In [ ]:
import seaborn as sns
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

In [ ]:
session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

In [ ]:
session.sql("CREATE OR REPLACE DATABASE SNOWPARK").collect()
session.sql("CREATE OR REPLACE SCHEMA SNOWPARK.TITANIC").collect()
session.sql("CREATE OR REPLACE STAGE ML_DATA").collect()

In [ ]:
titanic = sns.load_dataset("titanic")
titanic.columns = [c.upper() for c in titanic.columns]
titanic.to_csv("titanic.csv", index=False)

In [ ]:
session.file.put("titanic.csv", "@ml_data", overwrite=True)

In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
titanic_df = (
    session.read.option("infer_schema", True)
    .option("PARSE_HEADER", True)
    .csv("@ml_data/titanic.csv")
)
titanic_df.show()

In [ ]:
titanic_df.write.mode("overwrite").save_as_table("TITANIC")

In [ ]:
session.table("TITANIC").show()